## Bäume in Python

### Eigenschaften:
- nicht-linear
- dynamisch
- homogen (in den meisten Fällen, aber nicht immer)

#### Verhalten:
- Hierarchische Abhängigkeit von Knoten

#### Operatoren:
- Notwendige:
  - Erzeugen eines leeren Baums --> wird meistens nicht explizit aufgelistet
  - `add_child(...)`: An den aktuellen Knoten einen neuen Knoten als Kind anhängen
- Hilfreiche:
  - `traverse_tree_depth_first`: Traversiert durch Baum der Tiefe entlang &rarr; steigt so tief als möglich den Baum herab
  - `traverse_tree_breadth_first`: Traversiert durch Baum der Breite entlang &rarr; zuerst alle Elemente auf einem Niveau

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.children = []
        self.parent = None

    def add_child(self, child):
        child.parent = self
        self.children.append(child)
        return child

    def traverse_tree_depth_first(self):
        yield self
        for child in self.children:
            yield from child.traverse_tree_depth_first()

    def traverse_tree_breadth_first(self):
        #usually implemented with a queue, here some "trickery" so it works as a generator
        yield self
        for child in self.traverse_tree_breadth_first():
            if not child.children:
                return  # exit condition for leaf of tree --> stops infinite recursion
            yield from child.children
    
    def get_level(self):
        level = 0
        cur_parent = self.parent
        while cur_parent:
            cur_parent = cur_parent.parent
            level += 1
        return level         

    def print_tree(self):
        spaces = " " * (self.get_level() * 3 - 2)
        prefix = spaces + "└──" if self.parent else ""
        print(F"{prefix} {self.data}")
        if self.children:
            for child in self.children:
                child.print_tree()

    def __str__(self) -> str:
        return F"{self.data}"


In [2]:
current = Node("/")
root = current
usr_dir = root.add_child(Node("usr"))
etc_dir = root.add_child(Node("etc"))

bin_dir = usr_dir.add_child(Node("bin"))
games_dir = usr_dir.add_child(Node("games"))
lib_dir = usr_dir.add_child(Node("lib"))

dhcp_dir = etc_dir.add_child(Node("dhcp"))
systemd_dir = etc_dir.add_child(Node("systemd"))
cron_daily_dir = etc_dir.add_child(Node("cron.daily"))
cron_hourly_dir = etc_dir.add_child(Node("cron.hourly"))
cron_weekly_dir = etc_dir.add_child(Node("cron.weekly"))

mandb_dir = cron_daily_dir.add_child(Node("backup"))

current.print_tree()

 /
 └── usr
    └── bin
    └── games
    └── lib
 └── etc
    └── dhcp
    └── systemd
    └── cron.daily
       └── backup
    └── cron.hourly
    └── cron.weekly


### 🤓 Tiefe/Höhe von Bäumen
Sind Maße für die Länge eines Pfades von einem Knoten zur Wurzel und umgekehrt.  
Für die Tiefe (engl. level) wird der Wurzelknoten als Ursprung gewählt:
- Wurzel: 0
- Direkte Kinder: 1
- etc.
  
Für die Höhe (engl. height) des Baumes wird von den Blättern weg gezählt:
- gewähltes Blatt: 0
- Direktes Elternteil: 1
- etc.

In [9]:
print(F"{root.data}: {current.get_level()}")

/: 0


In [10]:
# Traverse depth wise for the first path, but print all levels
traveresed = False
start_node = root
while not traveresed:
    for child in start_node.children:
        print(F"{child.data}: {child.get_level()}")
    
    if start_node.children:
        start_node = start_node.children[0]
    else:
        traveresed = True

usr: 1
etc: 1
bin: 2
games: 2
lib: 2


---
## Traversieren von Bäumen
### Tiefensuche
Es wird so tief wie möglich in den Baum gegangen, bevor ein neuer Pfad eingeschlagen wird.

Wenn man hier präziser sein möchte, müsste noch zw. den unterschiedlichen Durchlaufrichtungen der Tiefensuche unterschieden werden.

In [4]:
for current in root.traverse_tree_depth_first():
    print(F"{current}: {current.get_level()}")

/: 0
usr: 1
bin: 2
games: 2
lib: 2
etc: 1
dhcp: 2
systemd: 2
cron.daily: 2
backup: 3
cron.hourly: 2
cron.weekly: 2


### Breitensuche
Es werden auf einem Niveau alle Knoten besucht, bevor tiefer in den Baum hinein gegangen wird.

In [3]:
for current in root.traverse_tree_breadth_first():
    print(F"{current}: {current.get_level()}")

/: 0
usr: 1
etc: 1
bin: 2
games: 2
lib: 2
dhcp: 2
systemd: 2
cron.daily: 2
cron.hourly: 2
cron.weekly: 2


---
### 🤓 Document Object Model für HTML
Ein Anwendungsfall von Bäumen als Datenstrukturen sind das Document Object Model, welches die Struktur bzw. das Interface für HTML- & XML-Dateien vorgibt.  
Hier werden wir die unten angeführte HTML-Datei parsen und mit unserer Baum-Datenstrukturen repräsentieren.

```html
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
<head>
    <title>simple</title>
</head>
<body>
<h1>A simple web page</h1>
<ul>
    <li>List item one</li>
    <li>List item two</li>
</ul>
<h2><a href="http://www.example.com">a link</a></h2>
</body>
</html>
```

Als Beispiel können wie die Website [hier](https://wiki.selfhtml.org/extensions/Selfhtml/frickl.php) darstellen.

In [13]:
html_string = "\
<html xmlns=\"http://www.w3.org/1999/xhtml\" xml:lang=\"en\" lang=\"en\"> \
<head> \
    <title>simple</title> \
</head> \
<body> \
<h1>A simple web page</h1> \
<ul> \
    <li>List item one</li> \
    <li>List item two</li> \
</ul> \
<h2><a href=\"http://www.example.com\">a link</a></h2> \
</body> \
</html> \
"

doc = Node("Document")
current = doc #start at document level

i = 0
while i < len(html_string):
    if html_string[i] == "<":
        if html_string[i+1] == "/":
            #tag end found
            current = current.parent
            i += 1
        else:
            #tag start found
            current = current.add_child(Node(""))

            #consume characters until end of this tag is reached to get its name/type
            start = i
            while html_string[i] != ">":
                i += 1
            end = i+1

            current.data = html_string[start:end]
    else:    
        #if no match consume next character
        i += 1

doc.print_tree()

 Document
 └── <html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
    └── <head>
       └── <title>
    └── <body>
       └── <h1>
       └── <ul>
          └── <li>
          └── <li>
       └── <h2>
          └── <a href="http://www.example.com">
